In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import load_model

# === Load Models with Architecture + Weights Already Defined ===
edge_model = load_model("simple_cnn_edge_model.h5")
mlp_model = load_model("mlp_environmental_model.h5")
lstm_model = load_model("lstm_sensor_model.h5")

# === Wrap EDGE CNN ===
edge_input = Input(shape=(224, 224, 1), name="edge_input")
edge_output = edge_model(edge_input)
edge_features = Dense(128, activation='relu', name="edge_fc")(edge_output)

# === Wrap MLP ===
mlp_input = Input(shape=(mlp_model.input_shape[1],), name="mlp_input")
mlp_output = mlp_model(mlp_input)
mlp_features = Dense(128, activation='relu', name="mlp_fc")(mlp_output)

# === Wrap LSTM ===
lstm_input = Input(shape=lstm_model.input_shape[1:], name="lstm_input")
lstm_output = lstm_model(lstm_input)
lstm_features = Dense(128, activation='relu', name="lstm_fc")(lstm_output)

# === Merge All ===
merged = Concatenate(name="merged_features")([edge_features, mlp_features, lstm_features])
x = Dense(64, activation='relu', name="dense_ensemble")(merged)
final_output = Dense(5, activation='softmax', name="final_output")(x)

# === Ensemble Model ===
ensemble_model = Model(
    inputs=[edge_input, mlp_input, lstm_input],
    outputs=final_output,
    name="edge_mlp_lstm_ensemble"
)

ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# === View Structure ===
ensemble_model.summary()


Model: "edge_mlp_lstm_ensemble"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ edge_input          │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 70)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_input          │ (None, 1, 15)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ edge_cnn_model      │ (None, 39)        │ 11,173,415 │ edge_input[0][0]  │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 70)        │     21,894 │ mlp_input[0][0]   │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 37)        │     10,661 │ lstm_input[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ edge_fc (Dense)     │ (None, 128)       │      5,120 │ edge_cnn_model[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_fc (Dense)      │ (None, 128)       │      9,088 │ functional_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_fc (Dense)     │ (None, 128)       │      4,864 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ merged_features     │ (None, 384)       │          0 │ edge_fc[0][0],    │
│ (Concatenate)       │                   │            │ mlp_fc[0][0],     │
│                     │                   │            │ lstm_fc[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_ensemble      │ (None, 64)        │     24,640 │ merged_features[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 5)         │        325 │ dense_ensemble[0… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,250,007 (42.92 MB)

 Trainable params: 11,250,007 (42.92 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def preprocess_edge_image(path):
    img = load_img(path, color_mode="grayscale", target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # normalize
    return np.expand_dims(img_array, axis=0)  # add batch dimension

# Example:
X_edge = preprocess_edge_image("processed_edges_dataset/Blueberry___healthy/image (100).png")


In [12]:
X_mlp = np.zeros((1, mlp_model.input_shape[1]))  # automatically matches (1, 70)
X_mlp[0, 5] = 1


In [13]:
# Example: manually create a single (1, 15)-dimensional sensor sample
sensor_values = np.array([[0.5]*15])  # dummy normalized values
X_lstm = np.expand_dims(sensor_values, axis=1)  # reshape to (1, 1, 15)


In [14]:
pred = ensemble_model.predict([X_edge, X_mlp, X_lstm])
print("Predicted class probabilities:", pred)
print("Predicted class:", np.argmax(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
Predicted class probabilities: [[0.19069815 0.19834596 0.20006137 0.2388833  0.1720112 ]]
Predicted class: 3


In [16]:
ensemble_model.save("final_ensemble_model.h5")

In [17]:
# If you used ImageDataGenerator before:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
gen = datagen.flow_from_directory('processed_edges_dataset', target_size=(224, 224))
class_names = list(gen.class_indices.keys())

print("Predicted class label:", class_names[3])


Found 55446 images belonging to 39 classes.
Predicted class label: Apple___healthy
